#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter ITP Audit Dashboard ( 2020 ) Parameters
Dashboard that shows performance metrics across browser to see the impact of ITP.
 1. Follow the instructions from <a href="https://docs.google.com/document/d/1HaRCMaBBEo0tSKwnofWNtaPjlW0ORcVHVwIRabct4fY/edit?usp=sharing" target="_blank">this document</a>
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'auth_write': 'service',  # Credentials used for writing data.
  'recipe_slug': 'ITP_Audit_Dashboard',  # BigQuery dataset for store dashboard tables.
  'recipe_name': '',  # Name of document to deploy to.
  'auth_read': 'user',  # Credentials used for reading data.
  'cm_account_id': '',  # Campaign Manager Account Id.
  'cm_advertiser_ids': '',  # Optional: Comma delimited list of CM advertiser ids.
  'floodlight_configuration_id': '',  # Floodlight Configuration Id for the Campaign Manager floodlight report.
  'dv360_partner_ids': '',  # Comma delimited list of DV360 Partner ids.
  'dv360_advertiser_ids': '',  # Optional: Comma delimited list of DV360 Advertiser ids.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute ITP Audit Dashboard ( 2020 )
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'drive': {
      'auth': 'user',
      'hour': [
      ],
      'copy': {
        'source': 'https://docs.google.com/spreadsheets/d/1rH_PGXOYW2mVdmAYnKbv6kcaB6lQihAyMsGtFfinnqg/',
        'destination': {'field': {'prefix': 'ITP Audit ','order': 1,'kind': 'string','description': 'Name of document to deploy to.','name': 'recipe_name','default': ''}}
      }
    }
  },
  {
    'dataset': {
      'auth': 'user',
      'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}}
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'timeout': 90,
        'body': {
          'metadata': {
            'dataRange': 'LAST_365_DAYS',
            'format': 'CSV',
            'title': {'field': {'prefix': 'ITP_Audit_Browser_','description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'kind': 'string'}}
          },
          'params': {
            'type': 'TYPE_GENERAL',
            'groupBys': [
              'FILTER_ADVERTISER',
              'FILTER_ADVERTISER_CURRENCY',
              'FILTER_MEDIA_PLAN',
              'FILTER_INSERTION_ORDER',
              'FILTER_LINE_ITEM',
              'FILTER_PAGE_LAYOUT',
              'FILTER_WEEK',
              'FILTER_MONTH',
              'FILTER_YEAR',
              'FILTER_PARTNER',
              'FILTER_LINE_ITEM_TYPE',
              'FILTER_DEVICE_TYPE',
              'FILTER_BROWSER'
            ],
            'metrics': [
              'METRIC_MEDIA_COST_ADVERTISER',
              'METRIC_IMPRESSIONS',
              'METRIC_CLICKS',
              'METRIC_TOTAL_CONVERSIONS',
              'METRIC_LAST_CLICKS',
              'METRIC_LAST_IMPRESSIONS',
              'METRIC_CM_POST_CLICK_REVENUE',
              'METRIC_CM_POST_VIEW_REVENUE',
              'METRIC_REVENUE_ADVERTISER'
            ]
          },
          'timezoneCode': {'field': {'description': 'Timezone for report dates.','kind': 'timezone','name': 'recipe_timezone','default': 'America/Los_Angeles'}}
        },
        'filters': {
          'FILTER_ADVERTISER': {
            'values': {'field': {'description': 'Optional: Comma delimited list of DV360 Advertiser ids.','name': 'dv360_advertiser_ids','order': 6,'default': '','kind': 'integer_list'}}
          },
          'FILTER_PARTNER': {
            'values': {'field': {'description': 'Comma delimited list of DV360 Partner ids.','name': 'dv360_partner_ids','order': 5,'default': '','kind': 'integer_list'}}
          }
        }
      },
      'delete': False
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'report': {
        'timeout': 90,
        'account': {'field': {'description': 'Campaign Manager Account Id.','name': 'cm_account_id','order': 2,'default': '','kind': 'string'}},
        'body': {
          'schedule': {
            'repeatsOnWeekDays': [
              'Sunday'
            ],
            'every': 1,
            'repeats': 'WEEKLY',
            'active': True
          },
          'kind': 'dfareporting#report',
          'floodlightCriteria': {
            'dateRange': {
              'kind': 'dfareporting#dateRange',
              'relativeDateRange': 'LAST_30_DAYS'
            },
            'reportProperties': {
              'includeUnattributedIPConversions': False,
              'includeUnattributedCookieConversions': True
            },
            'dimensions': [
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:site'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:floodlightAttributionType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:interactionType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:pathType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:browserPlatform'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:platformType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:week'
              }
            ],
            'floodlightConfigId': {
              'kind': 'dfareporting#dimensionValue',
              'dimensionName': 'dfa:floodlightConfigId',
              'matchType': 'EXACT',
              'value': {'field': {'description': 'Floodlight Configuration Id for the Campaign Manager floodlight report.','name': 'floodlight_configuration_id','order': 4,'default': '','kind': 'integer'}}
            },
            'metricNames': [
              'dfa:activityClickThroughConversions',
              'dfa:activityViewThroughConversions',
              'dfa:totalConversions',
              'dfa:totalConversionsRevenue'
            ]
          },
          'type': 'FLOODLIGHT',
          'name': {'field': {'prefix': 'ITP_Audit_Floodlight_','description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'kind': 'string'}},
          'format': 'CSV',
          'delivery': {
            'emailOwner': False
          }
        }
      },
      'out': {
        'bigquery': {
          'table': 'z_Floodlight_CM_Report',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}},
          'is_incremental_load': False
        }
      },
      'delete': False
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'report': {
        'timeout': 90,
        'account': {'field': {'description': 'Campaign Manager Account Id.','name': 'cm_account_id','order': 2,'default': '','kind': 'string'}},
        'body': {
          'schedule': {
            'repeatsOnWeekDays': [
              'Sunday'
            ],
            'every': 1,
            'repeats': 'WEEKLY',
            'active': True
          },
          'kind': 'dfareporting#report',
          'criteria': {
            'dimensions': [
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:campaign'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:campaignId'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:site'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:advertiser'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:advertiserId'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:browserPlatform'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:platformType'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:month'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'dfa:week'
              }
            ],
            'dateRange': {
              'kind': 'dfareporting#dateRange',
              'relativeDateRange': 'LAST_365_DAYS'
            },
            'dimensionFilters': [
            ],
            'metricNames': [
              'dfa:impressions',
              'dfa:clicks',
              'dfa:totalConversions',
              'dfa:activityViewThroughConversions',
              'dfa:activityClickThroughConversions'
            ]
          },
          'type': 'STANDARD',
          'fileName': {'field': {'prefix': 'ITP_Audit_Browser_','order': 1,'default': 'ITP_Audit_Dashboard_Browser','description': 'Name of the Campaign Manager browser report.','name': 'recipe_name','kind': 'string'}},
          'name': {'field': {'prefix': 'ITP_Audit_Browser_','order': 1,'default': 'ITP_Audit_Dashboard_Browser','description': 'Name of the Campaign Manager browser report.','name': 'recipe_name','kind': 'string'}},
          'format': 'CSV',
          'delivery': {
            'emailOwner': False
          }
        },
        'filters': {
          'dfa:advertiser': {
            'values': {'field': {'description': 'Optional: Comma delimited list of CM advertiser ids.','name': 'cm_advertiser_ids','order': 3,'default': '','kind': 'integer_list'}}
          }
        }
      },
      'out': {
        'bigquery': {
          'table': 'z_CM_Browser_Report_Dirty',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}},
          'is_incremental_load': False
        }
      },
      'delete': False
    }
  },
  {
    'sheets': {
      'sheet': {'field': {'prefix': 'ITP Audit ','order': 1,'kind': 'string','description': 'Name of document to deploy to.','name': 'recipe_name','default': ''}},
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_Environment',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}}
        }
      },
      'auth': 'user',
      'range': 'A:B',
      'header': True,
      'tab': 'Enviroment'
    }
  },
  {
    'sheets': {
      'sheet': {'field': {'prefix': 'ITP Audit ','order': 1,'kind': 'string','description': 'Name of document to deploy to.','name': 'recipe_name','default': ''}},
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_Browser',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}}
        }
      },
      'auth': 'user',
      'range': 'A:C',
      'header': True,
      'tab': 'Browser'
    }
  },
  {
    'sheets': {
      'sheet': {'field': {'prefix': 'ITP Audit ','order': 1,'kind': 'string','description': 'Name of document to deploy to.','name': 'recipe_name','default': ''}},
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_CM_Browser_lookup',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}}
        }
      },
      'auth': 'user',
      'range': 'A:C',
      'header': True,
      'tab': 'CM_Browser_lookup'
    }
  },
  {
    'sheets': {
      'sheet': {'field': {'prefix': 'ITP Audit ','order': 1,'kind': 'string','description': 'Name of document to deploy to.','name': 'recipe_name','default': ''}},
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_Device_Type',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}}
        }
      },
      'auth': 'user',
      'range': 'A:B',
      'header': True,
      'tab': 'Device_Type'
    }
  },
  {
    'sheets': {
      'sheet': {'field': {'prefix': 'ITP Audit ','order': 1,'kind': 'string','description': 'Name of document to deploy to.','name': 'recipe_name','default': ''}},
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_Floodlight_Attribution',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}}
        }
      },
      'auth': 'user',
      'range': 'A:B',
      'header': True,
      'tab': 'Floodlight_Attribution'
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'name': {'field': {'prefix': 'ITP_Audit_Browser_','description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'kind': 'string'}}
      },
      'out': {
        'bigquery': {
          'table': 'z_Dv360_Browser_Report_Dirty',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}}
        }
      }
    }
  },
  {
    'itp_audit': {
      'auth': 'user',
      'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','order': 1,'default': 'ITP_Audit_Dashboard','kind': 'string'}},
      'sheet': {'field': {'prefix': 'ITP Audit ','order': 1,'kind': 'string','description': 'Name of document to deploy to.','name': 'recipe_name','default': ''}},
      'account': {'field': {'description': 'Campaign Manager Account Id.','name': 'cm_account_id','order': 2,'default': '','kind': 'string'}},
      'timeout': 60
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
